# Import module 

Most codes from https://github.com/hwalsuklee/tensorflow-generative-model-collections/blob/master/CGAN.py

In [1]:
import os
import scipy
import random
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Loading MNIST data

In [2]:
os.environ['CUDA_VISIBLE_DEVICES']='2'
mnist = input_data.read_data_sets("data/mnist",one_hot=True)

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


# Discriminator Network

For easy reading，I did not put the "get_variable","conv2d" into one function

In [3]:
def discriminator(inputs, y, y_dim, batch_size, is_training=True, reuse=False):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : (64)4c2s-(128)4c2s_BL-FC1024_BL-FC1_S
    
    with tf.variable_scope("discriminator", reuse=reuse):
        
        '''concat "z" and "y" '''
        y = tf.reshape(y,shape=[batch_size,1,1,y_dim])
        inputs = tf.concat([inputs,y*tf.ones(inputs.shape)],axis=3)
        
        '''1st: Conv -> lrelu'''
        #conv
        shape = inputs.get_shape().as_list()
        w1 = tf.get_variable('d_wconv1', [4, 4, shape[-1], 64],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('d_bconv1', [64], initializer=tf.constant_initializer(0.0))
        net = tf.nn.conv2d(inputs, w1, strides=[1, 2, 2, 1], padding='SAME')
        net = tf.reshape(tf.nn.bias_add(net, b1), net.get_shape())
        #lrelu
        net = tf.maximum(net, 0.2*net)
        
        '''2nd: Conv -> bn -> lrelu'''
        #conv
        w2 = tf.get_variable('d_wconv2', [4, 4, 64, 128],initializer=tf.truncated_normal_initializer(stddev=0.02))
        b2 = tf.get_variable('d_bconv2', [128], initializer=tf.constant_initializer(0.0))
        net = tf.nn.conv2d(net, w2, strides=[1, 2, 2, 1], padding='SAME')
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                            is_training=is_training, scope='d_bn2')
        net = tf.reshape(tf.nn.bias_add(net, b2), net.get_shape())
        #lrelu
        net = tf.maximum(net, 0.2*net)
        
        net = tf.reshape(net, [batch_size, -1])
        
        '''3th: linear -> bn -> lrelu'''
        #linear
        shape = net.get_shape().as_list()        
        w3 = tf.get_variable("d_wlinear3", [shape[1], 1024], tf.float32,tf.random_normal_initializer(stddev=0.02))
        b3 = tf.get_variable("d_blinear3", [1024], initializer=tf.constant_initializer(0.0))
        net = tf.matmul(net, w3) + b3
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                        is_training=is_training, scope='d_bn3')
        #lrelu
        net = tf.maximum(net, 0.2*net)
        
        '''4th: linear '''
        #linear
        shape = net.get_shape().as_list()        
        w4 = tf.get_variable("d_wlinear4", [shape[1], 1], tf.float32,tf.random_normal_initializer(stddev=0.02))
        b4 = tf.get_variable("d_blinear4", [1], initializer=tf.constant_initializer(0.0))
        out_logit = tf.matmul(net, w4) + b4
        
        '''5th: sigmoid'''
        out = tf.nn.sigmoid(out_logit)        
        
        return out, out_logit


# Generator Network

In [4]:
def generator(z, y, y_dim, batch_size, is_training=True, reuse=False):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    
    with tf.variable_scope("generator", reuse=reuse):
        
        '''concat "z" and "y" '''
        z = tf.concat([z,y],axis=1)
        
        '''1st: linear -> bn -> relu '''
        #linear
        shape = z.get_shape().as_list()        
        w1 = tf.get_variable("g_wlinear1", [shape[1], 1024], tf.float32,tf.random_normal_initializer(stddev=0.02))
        b1 = tf.get_variable("g_blinear1", [1024], initializer=tf.constant_initializer(0.0))
        net = tf.matmul(z, w1) + b1
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                        is_training=is_training, scope='g_bn1')
        #relu
        net = tf.nn.relu(net)  
        
        '''2nd: linear -> bn -> relu '''
        #linear
        shape = net.get_shape().as_list()        
        w2 = tf.get_variable("g_wlinear2", [shape[1], 128*7*7], tf.float32,tf.random_normal_initializer(stddev=0.02))
        b2 = tf.get_variable("g_blinear2", [128*7*7], initializer=tf.constant_initializer(0.0))
        net = tf.matmul(net, w2) + b2
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                        is_training=is_training, scope='g_bn2')
        #relu
        net = tf.nn.relu(net) 
                     
        net = tf.reshape(net, [batch_size, 7, 7, 128])   
        
        '''3th: deconv -> bn -> relu '''
        #deconv
        output_shape = [batch_size, 14, 14, 64]
        w3 = tf.get_variable('g_wdeconv3', [4, 4, output_shape[-1], net.get_shape()[-1]],
                                           initializer=tf.random_normal_initializer(stddev=0.02))
        net = tf.nn.conv2d_transpose(net, w3, output_shape=output_shape, strides=[1, 2, 2, 1])
        b3 = tf.get_variable('g_bdeconv3', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        net = tf.reshape(tf.nn.bias_add(net, b3), net.get_shape())
        #bn
        net = tf.contrib.layers.batch_norm(net, decay=0.9,updates_collections=None, epsilon=1e-5,scale=True,
                                        is_training=is_training, scope='g_bn3')
        #relu
        net = tf.nn.relu(net)
        
        '''4th: deconv -> bn -> sigmoid '''
        #deconv
        output_shape = [batch_size, 28, 28, 1]
        w4 = tf.get_variable('g_wdeconv4', [4, 4, output_shape[-1], net.get_shape()[-1]],
                                           initializer=tf.random_normal_initializer(stddev=0.02))
        net = tf.nn.conv2d_transpose(net, w4, output_shape=output_shape, strides=[1, 2, 2, 1])
        b4 = tf.get_variable('g_bdeconv4', [output_shape[-1]], initializer=tf.constant_initializer(0.0))
        net = tf.reshape(tf.nn.bias_add(net, b4), net.get_shape())
        #sigmoid
        out = tf.nn.sigmoid(net)

        return out


# set the global parameters.

In [5]:
# some parameters
image_dims = [28, 28, 1]
batch_size = 64
z_dim = 62# dimension of noise-vector
y_dim = 10# dimension of condition-vector (label)
learning_rate = 0.0002
beta1 = 0.5

""" Graph Input """
# images
inputs = tf.placeholder(tf.float32, [batch_size] + image_dims, name='real_images')
# noises
z = tf.placeholder(tf.float32, [batch_size,z_dim], name='z')
#label
y = tf.placeholder(tf.float32, [batch_size,y_dim], name='y')

# Loss Function 

In [6]:
""" Loss Function """

# output of D for real images
D_real, D_real_logits = discriminator(inputs,y,y_dim, batch_size, is_training=True, reuse=False)

# output of D for fake images
G = generator(z,y,y_dim, batch_size, is_training=True, reuse=False)
D_fake, D_fake_logits = discriminator(G,y,y_dim, batch_size, is_training=True, reuse=True)

# get loss for discriminator
d_loss_real = tf.reduce_mean(
              tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real)))
d_loss_fake = tf.reduce_mean(
              tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake)))
d_loss = d_loss_real + d_loss_fake

# get loss for generator
g_loss = tf.reduce_mean(
         tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake)))


split the generator parameters and discriminator parameters into two list, then define how to train the two subnetwork and get the fake image for testing.

In [7]:
""" Training """
# divide trainable variables into a group for D and a group for G
t_vars = tf.trainable_variables()
d_vars = [var for var in t_vars if 'd_' in var.name]
g_vars = [var for var in t_vars if 'g_' in var.name]
# optimizers
d_optim = tf.train.AdamOptimizer(learning_rate, beta1=beta1).minimize(d_loss, var_list=d_vars)
g_optim = tf.train.AdamOptimizer(learning_rate*5, beta1=beta1).minimize(g_loss, var_list=g_vars)


"""" Testing """
# for test
fake_images = generator(z,y,y_dim,batch_size, is_training=False, reuse=True)
# graph inputs for visualize training results
sample_z = np.random.uniform(-1, 1, size=(batch_size , z_dim))
test_labels_onehot = np.zeros([batch_size, y_dim],dtype = np.float32)
test_labels_onehot[np.arange(batch_size), np.arange(batch_size)%int(np.sqrt(batch_size))] = 1.0

# Training a GAN

In [9]:
start_epoch = 0
start_batch_id = 0


num_steps = 25*1093
# loop for epoch
start_time = time.time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step_ind in range(num_steps):
    
    '''get the real data'''
    real_image_batch = mnist.train.next_batch(batch_size)
    batch_images,batch_labels = real_image_batch
    batch_images = batch_images.reshape([batch_size,28,28,1]).astype(np.float32)
    batch_labels = real_image_batch[1].astype(np.float32)
    
    '''get the noise data'''
    batch_z = np.random.uniform(-1, 1, [batch_size, z_dim]).astype(np.float32)

    # update D network
    _ , D_loss = sess.run([d_optim, d_loss], feed_dict={inputs: batch_images, z: batch_z, y:batch_labels})
    # update G network
    _, G_loss = sess.run([g_optim, g_loss], feed_dict={z: batch_z, y:batch_labels})

    # display training status
    print("Step: [%d] d_loss: %.8f, g_loss: %.8f" % (step_ind, D_loss, G_loss) )

    # save training results for every 300 steps
    if np.mod(step_ind, 300) == 0:

        samples = sess.run(fake_images, feed_dict={z:sample_z, y:test_labels_onehot})
        # put the "batch_size" images into one big canvas
        row = col = int(np.sqrt(batch_size))
        img = np.zeros( [row*28, col*28] )
        for i in range(row):
            for j in range(col):
                img[i*28:(i+1)*28,j*28:(j+1)*28] = samples[i*col+j, :, :, :].squeeze()
        #save the result      
        scipy.misc.imsave('{}.jpg'.format(step_ind),img)

Step: [0] d_loss: 1.40009022, g_loss: 0.99826944
Step: [1] d_loss: 1.72552574, g_loss: 0.89677227
Step: [2] d_loss: 1.40992284, g_loss: 0.90540326
Step: [3] d_loss: 1.46819162, g_loss: 0.84609568
Step: [4] d_loss: 1.24894619, g_loss: 0.87113923
Step: [5] d_loss: 1.24371088, g_loss: 0.87964052
Step: [6] d_loss: 1.21443510, g_loss: 0.88207918
Step: [7] d_loss: 1.21805739, g_loss: 0.88212138
Step: [8] d_loss: 1.17671442, g_loss: 0.90708995
Step: [9] d_loss: 1.19989598, g_loss: 0.90570670
Step: [10] d_loss: 1.19404364, g_loss: 0.92360544
Step: [11] d_loss: 1.21732521, g_loss: 0.93373692
Step: [12] d_loss: 1.23231149, g_loss: 0.90987146
Step: [13] d_loss: 1.21831501, g_loss: 0.90658522
Step: [14] d_loss: 1.24069071, g_loss: 0.88136482
Step: [15] d_loss: 1.18991911, g_loss: 0.86105031
Step: [16] d_loss: 1.19786644, g_loss: 0.85212272
Step: [17] d_loss: 1.17470574, g_loss: 0.86985350
Step: [18] d_loss: 1.19833338, g_loss: 0.84412229
Step: [19] d_loss: 1.19530249, g_loss: 0.86246091
Step: [20]

Step: [164] d_loss: 1.36629438, g_loss: 0.75276393
Step: [165] d_loss: 1.38012505, g_loss: 0.74062490
Step: [166] d_loss: 1.36098433, g_loss: 0.72897208
Step: [167] d_loss: 1.35254931, g_loss: 0.73139477
Step: [168] d_loss: 1.37290132, g_loss: 0.72484374
Step: [169] d_loss: 1.35688424, g_loss: 0.73914874
Step: [170] d_loss: 1.36999559, g_loss: 0.73009014
Step: [171] d_loss: 1.35513902, g_loss: 0.74348950
Step: [172] d_loss: 1.37044191, g_loss: 0.72724813
Step: [173] d_loss: 1.35233355, g_loss: 0.74685997
Step: [174] d_loss: 1.36758804, g_loss: 0.72448313
Step: [175] d_loss: 1.36089635, g_loss: 0.75275671
Step: [176] d_loss: 1.36715269, g_loss: 0.75579458
Step: [177] d_loss: 1.37264085, g_loss: 0.74844599
Step: [178] d_loss: 1.36631012, g_loss: 0.73894095
Step: [179] d_loss: 1.36499226, g_loss: 0.75739211
Step: [180] d_loss: 1.37179232, g_loss: 0.73850590
Step: [181] d_loss: 1.37264574, g_loss: 0.75780225
Step: [182] d_loss: 1.37444925, g_loss: 0.73419988
Step: [183] d_loss: 1.36276937,

Step: [329] d_loss: 1.36869562, g_loss: 0.73489213
Step: [330] d_loss: 1.36171722, g_loss: 0.73814321
Step: [331] d_loss: 1.36433578, g_loss: 0.74838614
Step: [332] d_loss: 1.37132585, g_loss: 0.74629909
Step: [333] d_loss: 1.37917280, g_loss: 0.72513688
Step: [334] d_loss: 1.37947607, g_loss: 0.73729455
Step: [335] d_loss: 1.39709735, g_loss: 0.73894697
Step: [336] d_loss: 1.38324237, g_loss: 0.73179907
Step: [337] d_loss: 1.39046454, g_loss: 0.74035692
Step: [338] d_loss: 1.37454796, g_loss: 0.74118274
Step: [339] d_loss: 1.37718201, g_loss: 0.73850775
Step: [340] d_loss: 1.37023020, g_loss: 0.74608618
Step: [341] d_loss: 1.37266147, g_loss: 0.74053222
Step: [342] d_loss: 1.37613666, g_loss: 0.73386502
Step: [343] d_loss: 1.36692262, g_loss: 0.74643618
Step: [344] d_loss: 1.37761676, g_loss: 0.73722291
Step: [345] d_loss: 1.37727261, g_loss: 0.74311143
Step: [346] d_loss: 1.38194942, g_loss: 0.74472809
Step: [347] d_loss: 1.37961614, g_loss: 0.74147046
Step: [348] d_loss: 1.37178874,

Step: [494] d_loss: 1.36731350, g_loss: 0.73770511
Step: [495] d_loss: 1.36282158, g_loss: 0.73427439
Step: [496] d_loss: 1.36673927, g_loss: 0.73700643
Step: [497] d_loss: 1.39362431, g_loss: 0.73428661
Step: [498] d_loss: 1.38390994, g_loss: 0.74055564
Step: [499] d_loss: 1.37060535, g_loss: 0.74619883
Step: [500] d_loss: 1.38117397, g_loss: 0.73554075
Step: [501] d_loss: 1.38509202, g_loss: 0.72610235
Step: [502] d_loss: 1.38347852, g_loss: 0.72458255
Step: [503] d_loss: 1.37994659, g_loss: 0.72944450
Step: [504] d_loss: 1.36194921, g_loss: 0.73943472
Step: [505] d_loss: 1.37511837, g_loss: 0.73374486
Step: [506] d_loss: 1.36967707, g_loss: 0.73337454
Step: [507] d_loss: 1.36203051, g_loss: 0.73332125
Step: [508] d_loss: 1.38010120, g_loss: 0.72979718
Step: [509] d_loss: 1.39214694, g_loss: 0.74214524
Step: [510] d_loss: 1.37631035, g_loss: 0.74509859
Step: [511] d_loss: 1.38630176, g_loss: 0.72970295
Step: [512] d_loss: 1.37038577, g_loss: 0.73742807
Step: [513] d_loss: 1.38394308,

Step: [659] d_loss: 1.36191142, g_loss: 0.74152279
Step: [660] d_loss: 1.37355161, g_loss: 0.73949313
Step: [661] d_loss: 1.37365937, g_loss: 0.73133105
Step: [662] d_loss: 1.38137257, g_loss: 0.71456140
Step: [663] d_loss: 1.37168276, g_loss: 0.72430110
Step: [664] d_loss: 1.38675523, g_loss: 0.72747904
Step: [665] d_loss: 1.39509892, g_loss: 0.72898579
Step: [666] d_loss: 1.37925029, g_loss: 0.73456383
Step: [667] d_loss: 1.36448836, g_loss: 0.74542522
Step: [668] d_loss: 1.38926625, g_loss: 0.73370039
Step: [669] d_loss: 1.37876630, g_loss: 0.74944520
Step: [670] d_loss: 1.37493539, g_loss: 0.73906958
Step: [671] d_loss: 1.37104797, g_loss: 0.74836522
Step: [672] d_loss: 1.38939571, g_loss: 0.73932040
Step: [673] d_loss: 1.38541591, g_loss: 0.74052501
Step: [674] d_loss: 1.39761853, g_loss: 0.75083411
Step: [675] d_loss: 1.38892579, g_loss: 0.73605883
Step: [676] d_loss: 1.36834025, g_loss: 0.74231935
Step: [677] d_loss: 1.37556338, g_loss: 0.72770417
Step: [678] d_loss: 1.37018192,

...

Step: [26580] d_loss: 1.38634992, g_loss: 0.69485891
Step: [26581] d_loss: 1.38624883, g_loss: 0.69372261
Step: [26582] d_loss: 1.38629317, g_loss: 0.69167966
Step: [26583] d_loss: 1.38620448, g_loss: 0.69237739
Step: [26584] d_loss: 1.38609862, g_loss: 0.69292098
Step: [26585] d_loss: 1.38626146, g_loss: 0.69363421
Step: [26586] d_loss: 1.38620925, g_loss: 0.69353700
Step: [26587] d_loss: 1.38620877, g_loss: 0.69321883
Step: [26588] d_loss: 1.38688421, g_loss: 0.69098938
Step: [26589] d_loss: 1.38638759, g_loss: 0.69222337
Step: [26590] d_loss: 1.38622665, g_loss: 0.69460773
Step: [26591] d_loss: 1.38661551, g_loss: 0.69439399
Step: [26592] d_loss: 1.38655305, g_loss: 0.69500017
Step: [26593] d_loss: 1.38635516, g_loss: 0.69314682
Step: [26594] d_loss: 1.38636923, g_loss: 0.69143307
Step: [26595] d_loss: 1.38715291, g_loss: 0.69080424
Step: [26596] d_loss: 1.38702583, g_loss: 0.68941963
Step: [26597] d_loss: 1.38629270, g_loss: 0.69165862
Step: [26598] d_loss: 1.38632572, g_loss: 0.69

Step: [26735] d_loss: 1.38601947, g_loss: 0.69454145
Step: [26736] d_loss: 1.38709128, g_loss: 0.69522530
Step: [26737] d_loss: 1.38858008, g_loss: 0.69647980
Step: [26738] d_loss: 1.38951600, g_loss: 0.69759572
Step: [26739] d_loss: 1.38809276, g_loss: 0.70254481
Step: [26740] d_loss: 1.38670874, g_loss: 0.69619030
Step: [26741] d_loss: 1.38644099, g_loss: 0.69470620
Step: [26742] d_loss: 1.38649654, g_loss: 0.69193357
Step: [26743] d_loss: 1.38653946, g_loss: 0.68949986
Step: [26744] d_loss: 1.38636398, g_loss: 0.69120187
Step: [26745] d_loss: 1.38652480, g_loss: 0.69147933
Step: [26746] d_loss: 1.38654411, g_loss: 0.69503832
Step: [26747] d_loss: 1.38640833, g_loss: 0.69501829
Step: [26748] d_loss: 1.38623953, g_loss: 0.69470990
Step: [26749] d_loss: 1.38623428, g_loss: 0.69364548
Step: [26750] d_loss: 1.38620615, g_loss: 0.69184846
Step: [26751] d_loss: 1.38625133, g_loss: 0.69222414
Step: [26752] d_loss: 1.38618708, g_loss: 0.69356489
Step: [26753] d_loss: 1.38617885, g_loss: 0.69

Step: [26890] d_loss: 1.38634646, g_loss: 0.69286847
Step: [26891] d_loss: 1.38627768, g_loss: 0.69380468
Step: [26892] d_loss: 1.38649046, g_loss: 0.69531965
Step: [26893] d_loss: 1.38624465, g_loss: 0.69393349
Step: [26894] d_loss: 1.38618517, g_loss: 0.69237900
Step: [26895] d_loss: 1.38626599, g_loss: 0.69213212
Step: [26896] d_loss: 1.38626122, g_loss: 0.69288313
Step: [26897] d_loss: 1.38628531, g_loss: 0.69364810
Step: [26898] d_loss: 1.38632989, g_loss: 0.69311213
Step: [26899] d_loss: 1.38626122, g_loss: 0.69283772
Step: [26900] d_loss: 1.38631773, g_loss: 0.69284296
Step: [26901] d_loss: 1.38624561, g_loss: 0.69298226
Step: [26902] d_loss: 1.38619471, g_loss: 0.69334745
Step: [26903] d_loss: 1.38628435, g_loss: 0.69360530
Step: [26904] d_loss: 1.38628829, g_loss: 0.69240654
Step: [26905] d_loss: 1.38615346, g_loss: 0.69242960
Step: [26906] d_loss: 1.38629210, g_loss: 0.69325638
Step: [26907] d_loss: 1.38634300, g_loss: 0.69332397
Step: [26908] d_loss: 1.38662851, g_loss: 0.69

Step: [27045] d_loss: 1.38564348, g_loss: 0.69748926
Step: [27046] d_loss: 1.38631248, g_loss: 0.69334149
Step: [27047] d_loss: 1.38627172, g_loss: 0.69252026
Step: [27048] d_loss: 1.38619578, g_loss: 0.69184518
Step: [27049] d_loss: 1.38617074, g_loss: 0.69282138
Step: [27050] d_loss: 1.38626289, g_loss: 0.69304037
Step: [27051] d_loss: 1.38627386, g_loss: 0.69359332
Step: [27052] d_loss: 1.38625383, g_loss: 0.69333148
Step: [27053] d_loss: 1.38629961, g_loss: 0.69314420
Step: [27054] d_loss: 1.38612962, g_loss: 0.69287395
Step: [27055] d_loss: 1.38628495, g_loss: 0.69319856
Step: [27056] d_loss: 1.38631058, g_loss: 0.69331789
Step: [27057] d_loss: 1.38601017, g_loss: 0.69315422
Step: [27058] d_loss: 1.38627839, g_loss: 0.69301426
Step: [27059] d_loss: 1.38656509, g_loss: 0.69412196
Step: [27060] d_loss: 1.38635349, g_loss: 0.69284570
Step: [27061] d_loss: 1.38641667, g_loss: 0.69390643
Step: [27062] d_loss: 1.38665390, g_loss: 0.69385791
Step: [27063] d_loss: 1.38661563, g_loss: 0.69

Step: [27200] d_loss: 1.38411283, g_loss: 0.69021827
Step: [27201] d_loss: 1.38632858, g_loss: 0.69374907
Step: [27202] d_loss: 1.38641286, g_loss: 0.69262081
Step: [27203] d_loss: 1.38632226, g_loss: 0.69153607
Step: [27204] d_loss: 1.38631725, g_loss: 0.69182301
Step: [27205] d_loss: 1.38631010, g_loss: 0.69252640
Step: [27206] d_loss: 1.38625669, g_loss: 0.69332248
Step: [27207] d_loss: 1.38631690, g_loss: 0.69364583
Step: [27208] d_loss: 1.38625395, g_loss: 0.69331884
Step: [27209] d_loss: 1.38627577, g_loss: 0.69308233
Step: [27210] d_loss: 1.38762307, g_loss: 0.69207770
Step: [27211] d_loss: 1.38624668, g_loss: 0.69395047
Step: [27212] d_loss: 1.38631010, g_loss: 0.69304001
Step: [27213] d_loss: 1.38629127, g_loss: 0.69288641
Step: [27214] d_loss: 1.38625753, g_loss: 0.69296455
Step: [27215] d_loss: 1.38626814, g_loss: 0.69307542
Step: [27216] d_loss: 1.38620901, g_loss: 0.69295812
Step: [27217] d_loss: 1.38744402, g_loss: 0.69165003
Step: [27218] d_loss: 1.38623357, g_loss: 0.69

<center/>one of the result image</center>